In [40]:
import argparse
import numpy as np
import os
from chunkchromatin.simulation import Simulation
from chunkchromatin.chromosome import Chromosome
from chunkchromatin.lamina import Lamina
from chunkchromatin.hdf5_format import HDF5Reporter
from chunkchromatin.simulation import EKExceedsError
from chunkchromatin.condensate import Condensate
import openmm as mm
from initial_conformations import create_random_walk

import json

In [41]:
import numpy as np

def generate_pattern_blocks():
    """
    Generate monomer_types array for 120 monomers with fixed block pattern:
    4 repetitions of {10*A, 10*B, 10*C}.
    A=0, B=1, C=2.
    """
    monomer_types = []
    for _ in range(4):
        monomer_types.extend([0]*10)  # A block
        monomer_types.extend([1]*10)  # B block
        monomer_types.extend([2]*10)  # C block
    return np.array(monomer_types, dtype=int)

def generate_condensate_positions(n_condensates, chromatin_positions, box_length, min_dist=1.0, seed=42):
    np.random.seed(seed)
    condensate_positions = []
    while len(condensate_positions) < n_condensates:
        trial_pos = np.random.uniform(0, box_length, size=3)
        dists = np.linalg.norm(chromatin_positions - trial_pos, axis=1)
        if np.all(dists > min_dist):
            condensate_positions.append(trial_pos)
    return np.array(condensate_positions)



In [42]:
# === Parameters ===
n_chromatin = 120   # divisible by 3 for equal abundance
n_condensates = 10
N = n_chromatin + n_condensates
density = 0.33
box_length = (N / density) ** (1 / 3.)
chains = [(0, n_chromatin, False)]   # single chain of 120 chromatin beads

# === Chromatin & Condensate Types ===
monomer_types = generate_pattern_blocks()
np.save("monomer_types_cond.npy", monomer_types)

condensate_types = np.random.choice([0, 1], size=n_condensates, replace=True)
chromatin_types_full = np.concatenate([monomer_types, [-1] * n_condensates])

print("N =", N)
print("len(monomer_types) =", len(monomer_types))
print("len(chromatin_types_full) =", len(chromatin_types_full))
print("len(condensate_types) =", len(condensate_types))

# === Interaction Matrices ===
interaction_matrix = np.array([
    [0.05, 0.05, 0.08],
    [0.05, 0.13, 0.17],
    [0.08, 0.17, 0.22]
])

epsilon_cc = np.array([
    [1.0, 0.8],
    [0.8, 1.2]
])

epsilon_cchr = np.array([
    [0.2, 0.3, 0.4],
    [0.1, 0.5, 0.6]
])

# === Reporter & Simulation Setup ===
out_dir = 'test_output_condensate_1'
os.makedirs(out_dir, exist_ok=True)
reporter = HDF5Reporter(folder=out_dir, max_data_length=500, overwrite=True)

sim = Simulation(
    integrator_type="variableLangevin",
    temperature=300.0,  # Kelvin
    gamma=0.05,         # 1/ps
    timestep=5,         # fs
    platform='CPU',
    N=N,
    reporter=reporter
)

# === Force Objects ===
chromosome = Chromosome(N, chains, sim)
lamina = Lamina(N, chains, sim)

# Add condensate–condensate and condensate–chromatin forces
condensate = Condensate(
    N=N,
    chains=chains,
    simulation=sim,
    condensate_types=condensate_types,
    chromatin_types=chromatin_types_full,
    epsilon_cc=epsilon_cc,
    epsilon_cchr=epsilon_cchr,
    cutoff=3.0,
    alpha=6
)

# === Initial Positions ===
chromatin_positions = create_random_walk(step_size=1, N=n_chromatin)
condensate_positions = generate_condensate_positions(
    n_condensates=n_condensates,
    chromatin_positions=chromatin_positions,
    box_length=box_length,
    min_dist=1.0
)

positions = np.vstack([chromatin_positions, condensate_positions])
sim.set_positions(positions)

# === Add Chromatin Forces ===
harmonic_bond_force = chromosome.add_harmonic_bond()
angle_force = chromosome.add_angle_force()
nonbonded_pair_potential_force = chromosome.add_nonbonded_pair_potential(
    sim,
    interaction_matrix,
    np.concatenate([monomer_types, [-1] * n_condensates])
)
spherical_confinement_force = lamina.add_spherical_confinement(sim)

sim.add_force(harmonic_bond_force)
sim.add_force(angle_force)
sim.add_force(nonbonded_pair_potential_force)
sim.add_force(spherical_confinement_force)

# === Finalize System ===
sim.create_context()
sim.set_velocities()


# Previous System Setup 
# n_chromatin = 100
# n_condensates = 10
# N = n_chromatin + n_condensates
# density = 0.33
# box_length = (N / density) ** (1 / 3.)
# chains = [(0, 50, False), (50, 100, False)]

#use the function above to generate 40 monomers of each type 

## this is the previous method to generate monomers
#monomer_types = generate_block_copolymer(n_chromatin, min_block_size=5, types=(0, 1, 2), seed=42)
#np.save("monomer_types.npy", monomer_types)



N = 130
len(monomer_types) = 120
len(chromatin_types_full) = 130
len(condensate_types) = 10


In [39]:
import numpy as np

chromatin_types_full = np.load("C:/Users/arnav/Personal - Arnav Chhajed/Northwestern/genome_organization/examples/arnav/monomer_types.npy")

n_type0 = np.sum(chromatin_types_full == 0)
n_type1 = np.sum(chromatin_types_full == 1)
n_type2 = np.sum(chromatin_types_full == 2)
n_condensates = np.sum(chromatin_types_full == -1)

print(f"Type 0 monomers: {n_type0}")
print(f"Type 1 monomers: {n_type1}")
print(f"Type 2 monomers: {n_type2}")
print(f"Condensates: {len(condensate_types)}")
print(f"Total particles: {len(chromatin_types_full)}")



Type 0 monomers: 40
Type 1 monomers: 40
Type 2 monomers: 40
Condensates: 10
Total particles: 120


In [37]:
import numpy as np

n_chromatin = 120
n_condensates = 10

# load your existing chromatin-only monomer types
monomer_types = np.load("C:/Users/arnav/Personal - Arnav Chhajed/Northwestern/genome_organization/examples/arnav/monomer_types.npy")

# sanity check
print("Chromatin monomers loaded:", len(monomer_types))

# extend with -1 for condensates
chromatin_types_full = np.concatenate([monomer_types, [-1]*n_condensates])

# save
np.save("chromatin_types_full.npy", chromatin_types_full)

print("Saved chromatin_types_full.npy with shape:", chromatin_types_full.shape)


Chromatin monomers loaded: 120
Saved chromatin_types_full.npy with shape: (130,)


In [43]:
# === Run Simulation ===
for _ in range(200):  # 10 blocks of 1000 steps each
    sim.run_simulation_block(1000)
    with open(f"{out_dir}/simulation_stats.txt", "a") as f:
        stats = str(sim.print_stats())
        f.write(stats + "\n")
    reporter.dump_data()


INFO:root:block    0 pos[1]=[1.3 2.6 -0.4] t=0.0ns kin=4.76 pot=4.04 SPS=997



Simulation Statistics:
Current block: 1
Total steps: 1000
Simulation time: 8.07τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.01 kT/particle


INFO:root:block    1 pos[1]=[0.8 2.1 2.9] t=0.0ns kin=3.43 pot=3.36 SPS=979



Simulation Statistics:
Current block: 2
Total steps: 2000
Simulation time: 15.98τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.01 kT/particle


INFO:root:block    2 pos[1]=[4.0 0.6 1.5] t=0.0ns kin=2.74 pot=2.64 SPS=941


...


Simulation Statistics:
Current block: 198
Total steps: 198000
Simulation time: 1942.63τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.00 kT/particle


INFO:root:block  198 pos[1]=[-1.0 -1.3 -3.6] t=2.0ns kin=1.47 pot=1.66 SPS=1499



Simulation Statistics:
Current block: 199
Total steps: 199000
Simulation time: 1952.26τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.00 kT/particle


INFO:root:block  199 pos[1]=[-4.3 2.1 0.6] t=2.0ns kin=1.51 pot=1.48 SPS=1563



Simulation Statistics:
Current block: 200
Total steps: 200000
Simulation time: 1962.31τ
Kinetic energy: 0.00 kT/particle
Potential energy: 0.00 kT/particle
Total energy: 0.00 kT/particle
